In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [7]:
import csv
import getpass
from datetime import datetime as dt
from datetime import timedelta as td
import numpy as np
import math
import os
import pandas as pd

login = False

while login == False:
    # A log in which uses a databases to find multiple usernames and passwords
    data=[]
    with open("Users.csv") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            data.append(row)
    #print(data)
    print("Welcome to SmartForecaster! \n")
    username = input("Please enter username: ")
    password = getpass.getpass("Please enter password: ")
    #loops through all the data in column 0 and 1 assigns it to a variable called 'colA/colB'
    colA = [x[0] for x in data]
    colB = [x[1] for x in data]
    #print(colA)
    #print(ColB)
    if username in colA:
        #iterates through the list for the length of the list
        for k in range (0, len(colA)):
            #checks if username matches the password in the a joining column
            if colA[k] == username and colB[k] == password:
                print("You are logged in.\n")
                print("Enter 1 to Forecast.")
                print("Enter 2 to Update Historical Data.")
                print("Enter 3 to Logout")
                print("")
                login = True
                option_input = int(input())
                if (option_input == 1):
                    print("Welcome to Forecast Page")
                    # Get start date input and validate date format
                    print('Please enter the start date of forecast (YYYY-MM-DD):')
                    # Initialize a value for startDate
                    startDate = 'some date'
                    while not startDate == '':
                        startDate = input()
                        # Validate date format as YYYY-MM-DD
                        try:
                            startDate = dt.strptime(startDate,'%Y-%m-%d')
                            break
                        # Prompt user to input in correct format
                        except ValueError:
                            print('Please enter date in format YYYY-MM-DD:')
                            continue 
                    #Create a list for date
                    dateList = []
                    #Add startDate to list
                    dateList.append(startDate)
                    #Initialize date variable as start date
                    date = startDate
                    #Add 6 more days into list
                    while len(dateList)<7:
                        addDate = date+td(days=1)
                        date=addDate
                        dateList.append(addDate)
                    #Convert List to dictionary and initialize all values to 0
                    dateArrival ={i:0 for i in dateList}
                    #Enter arrival for 7 days as key
                    for i in dateArrival.keys():
                        print('Please enter arrival for the date '+str(i))
                        arrival = int(input())
                        dateArrival[i]=arrival 

                    # Convert dateArrival into a dateframe 
                    arrivalTY = pd.DataFrame(list(dateArrival.items()), columns=['Date', 'New Arrival'])

                    # Break total arrival into different shifts for each day with ratio
                    # 20% bookings check in during day shift, 70% check in during swing and 10% check in during grave
                    arrivalTY = arrivalTY.assign(Day_Arrival = arrivalTY['New Arrival']*0.2)
                    arrivalTY = arrivalTY.assign(Swing_Arrival = arrivalTY['New Arrival']*0.7)
                    arrivalTY = arrivalTY.assign(Grave_Arrival = arrivalTY['New Arrival']*0.1)

                    # Break date tinto year and date
                    arrivalTY['Year'] = arrivalTY['Date'].dt.year  
                    arrivalTY['Date'] = arrivalTY['Date'].dt.strftime('%m-%d')

                    # Read the csv historical file for the data
                    arrivalLY = pd.read_csv('Historical Arrival Data.csv') 
                    # Fill the missing data with 0
                    arrivalLY = arrivalLY.fillna(0)
                    # Convert Date into datetime format
                    arrivalLY['Date'] = pd.to_datetime(arrivalLY['Date'],format='%Y-%m-%d')
                    # Break date into year and date
                    arrivalLY['Year'] = arrivalLY['Date'].dt.year 
                    arrivalLY['Date'] = arrivalLY['Date'].dt.strftime('%m-%d')
                    # Remove outliers
                    # Select data about the same date last year only
                    arrivalLY = arrivalLY[arrivalLY['Date'].isin(list(arrivalTY['Date']))]
                    # To select data of the previous year only
                    # Extract this year forecast into a list
                    yearForecast = list(arrivalTY['Year'])
                    # Subtract 1 from each variable in list
                    compareYear = [x-1 for x in yearForecast]
                    # Select year data that match year data in compareYear from data of last year arrival
                    arrivalLY = arrivalLY[arrivalLY['Year'].isin(compareYear)] 
                    # Merge 2 dataframes of same date this year and last year
                    combinedData = pd.merge(arrivalTY,arrivalLY, on = 'Date', how = 'left')
                    # Calculate the difference of this year data and last year data for each shift
                    combinedData['day Diff'] = combinedData['Day_Arrival'] - combinedData['Arrival of guests during day']
                    combinedData['swing Diff'] = combinedData['Swing_Arrival'] - combinedData['Arrival of guests during swing']
                    combinedData['grave Diff'] = combinedData['Grave_Arrival'] - combinedData['Arrival of guests during grave']
                    # Forecast labor for day shift
                    shift = ['day','swing','grave']
                    for s in range(len(shift)): 
                        combinedData['Expected '+shift[s]+' Staff'] = combinedData['Amount of workers at '+shift[s]]+round((combinedData[shift[s]+' Diff']-100)/50)
                    # Select Columns to print for viewers
                    columnsToSee = ['Date','Total_Arrival','New Arrival','Year_x','Amount of workers at day','Amount of workers at swing','Amount of workers at grave','day Diff','swing Diff','grave Diff','Expected day Staff','Expected swing Staff','Expected grave Staff']
                    combinedData = combinedData.loc[:,columnsToSee]
                    from IPython.display import display
                    display(combinedData)

                    while True :

                        print('Enter an option from the below content ( -1 to exit editing ):')
                        print('1. Modify Day staff')
                        print('2. Modify Swing staff')
                        print('3. Modify Grave staff')
                        inp = int(input())
                        if (inp == 1):
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            print('Enter the value:')
                            val = float(input())
                            combinedData.set_value(index, 'Expected day Staff', val)

                        elif (inp == 2):
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            print('Enter the value:')
                            val = float(input())
                            combinedData.set_value(index, 'Expected swing Staff', val)

                        elif (inp == 3):
                            print('Enter the index of the day which you want to edit:')
                            index = int(input())
                            print('Enter the value:')
                            val = float(input())
                            combinedData.set_value(index, 'Expected grave Staff', val)

                        elif (inp == -1):
                            break
                        
                        else:
                            print('Please check the number and retry')



                    fileName = 'Prediction Data.xlsx'
                    hourlyPay = 18
                    fullTime = 8

                    combinedData.to_excel(fileName, sheet_name='Latest', index = False)

                    totalDay = combinedData['Expected day Staff'].sum()
                    totalSwing = combinedData['Expected swing Staff'].sum()
                    totalGrave = combinedData['Expected grave Staff'].sum()
                    totalStaff = totalDay+totalSwing+totalGrave
                    print('Total employees required for the week '+str(totalStaff))
                    def labor_Expense(rate,hour,staff):
                        laborExpense = rate*hour*staff
                        return laborExpense
                    print('Labor expenses for the week is estimated at $'+str(labor_Expense(hourlyPay,fullTime,totalStaff)))
                
                elif (option_input == 2):
                    # Prompt user for filename
                    fileName = input('Please enter the file name for uploading (include csv ending):')

                    # Read new_data csv for new data, set first column Date to be index
                    newData = pd.read_csv(fileName, index_col=[0]) 

                    # Read historical data csv, set first column Date to be index
                    historicalData = pd.read_csv('Historical Arrival Data.csv',index_col=[0])

                    # Use reindex and unite the new data to historical data
                    historicalData=historicalData.reindex(historicalData.index.union(newData.index))

                    # In case data are overlap in both dataframes, change all overlap data into missing data
                    historicalData.loc[historicalData.index.intersection(newData.index),:] = np.nan

                    # Fill all missing data in historical dataframe with values in newData dataframe
                    historicalData = historicalData.combine_first(newData)
                    display(historicalData)
                    # Export dataframe back to csv file
                    historicalData.to_csv('Historical Arrival Data.csv')
                
                else:
                    print('You are successfully logged out.')
                    break
    else:
        print("\n Wrong username or password. Please try again.\n")




Welcome to SmartForecaster! 

Please enter username: manae1
Please enter password: ········

 Wrong username or password. Please try again.

Welcome to SmartForecaster! 

Please enter username: manager1
Please enter password: ········
You are logged in.

Enter 1 to Forecast.
Enter 2 to Update Historical Data.
Enter 3 to Logout

1
Welcome to Forecast Page
Please enter the start date of forecast (YYYY-MM-DD):
2019-12-12
Please enter arrival for the date 2019-12-12 00:00:00
1
Please enter arrival for the date 2019-12-13 00:00:00
1
Please enter arrival for the date 2019-12-14 00:00:00
1
Please enter arrival for the date 2019-12-15 00:00:00
1
Please enter arrival for the date 2019-12-16 00:00:00
1
Please enter arrival for the date 2019-12-17 00:00:00
1
Please enter arrival for the date 2019-12-18 00:00:00
1


Date  Total_Arrival  New Arrival  Year_x  Amount of workers at day  \
0  12-12          702.0            1    2019                       4.0   
1  12-13          832.0            1    2019                       5.0   
2  12-14         1113.0            1    2019                       6.0   
3  12-15          345.0            1    2019                       3.0   
4  12-16          427.0            1    2019                       3.0   
5  12-17          456.0            1    2019                       4.0   
6  12-18          325.0            1    2019                       3.0   

   Amount of workers at swing  Amount of workers at grave  day Diff  \
0                        11.0                         5.0    -123.8   
1                        11.0                         6.0    -177.8   
2                        12.0                         5.0    -255.8   
3                         6.0                         2.0    -100.8   
4                         4.0                         2.0    -149.8   
5                         5.0                         3.0    -164.8   
6                         4.0                         2.0    -111.8   

   swing Diff  grave Diff  Expected day Staff  Expected swing Staff  \
0      -431.3      -145.9                 0.0                   0.0   
1      -431.3      -221.9                -1.0                   0.0   
2      -588.3      -267.9                -1.0                  -2.0   
3      -177.3       -65.9                -1.0                   0.0   
4      -148.3      -127.9                -2.0                  -1.0   
5      -215.3       -74.9                -1.0                  -1.0   
6      -155.3       -56.9                -1.0                  -1.0   

   Expected grave Staff  
0                   0.0  
1                   0.0  
2                  -2.0  
3                  -1.0  
4                  -3.0  
5                   0.0  
6                  -1.0

Enter an option from the below content ( -1 to exit editing ):
1. Modify Day staff
2. Modify Swing staff
3. Modify Grave staff
1
Enter the index of the day which you want to edit:
0
Enter the value:
2


/Users/PhuongMPham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:135: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Enter an option from the below content ( -1 to exit editing ):
1. Modify Day staff
2. Modify Swing staff
3. Modify Grave staff
-1
Total employees required for the week -17.0
Labor expenses for the week is estimated at $-2448.0
